<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_EGLMS_Project/blob/main/%5BExp_13%5DMovie_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

## 1. Import 및 라이브러리 다운로드

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 277 kB/s 


In [2]:
import numpy as np
import scipy
import implicit
import pandas as pd

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


## 2. 데이터 준비와 전처리

In [3]:
rating_file_path = '/content/drive/MyDrive/인공지능/아이펠/ES/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [ ]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
movie_file_path= '/content/drive/MyDrive/인공지능/아이펠/ES/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 데이터 프레임 합치기
- https://yganalyst.github.io/data_handling/Pd_12/
  - 데이터에 존재하는 고유값을 기준으로 병합하는 merge() 사용
    - 기준은 둘다 공통으로 있는 movie_id로 지정하여 병합
    - how는 outter로 주어 어느 한쪽에라도 없는 데이터가 있는 경우 NaN값이 지정되게 함

In [11]:
movies_data = pd.merge(ratings, movies, how='outer', on='movie_id' )

In [12]:
movies_data

,user_id,movie_id,ratings,timestamp,title,genre
0,1.0,1193,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836728,NaN,3829,NaN,NaN,Mad About Mambo (2000),Comedy|Romance
836729,NaN,3856,NaN,NaN,Autumn Heart (1999),Drama
836730,NaN,3891,NaN,NaN,Turn It Up (2000),Crime|Drama
836731,NaN,3904,NaN,NaN,"Uninvited Guest, An (2000)",Drama


In [14]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836733 entries, 0 to 836732
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    836478 non-null  float64
 1   movie_id   836733 non-null  int64  
 2   ratings    836478 non-null  float64
 3   timestamp  836478 non-null  float64
 4   title      836733 non-null  object 
 5   genre      836733 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 44.7+ MB


In [17]:
movies_data.describe()

,user_id,movie_id,ratings,timestamp
count,836478.000000,836733.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.105342,3.958293,9.721628e+08
std,1729.255651,1091.865588,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


In [13]:
movies_data.isnull().sum()

user_id      255
movie_id       0
ratings      255
timestamp    255
title          0
genre          0
dtype: int64

- user_id와 ratings, timestamp가 모두 255개의 결측치를 가지고 있다.
- 전체 데이터셋에 비해 굉장히 미미한 데이터량이니 제거하도록 한다.

In [20]:
movies_data.dropna(inplace=True)

In [22]:
movies_data.isnull().sum()

user_id      0
movie_id     0
ratings      0
timestamp    0
title        0
genre        0
dtype: int64

## 3. 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [23]:
movies_data['movie_id'].nunique()

3628

In [24]:
movies_data['user_id'].nunique()

6039

In [26]:
genre_count = movies_data.groupby('title')['user_id'].count()
genre_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 4. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [ ]:
my_favorite = ['Matrix, The ' , 'Men in Black' ,'Toy Story' ,'City of Lost Children, The' ,'Sudden Death']

my_playlist = pd.DataFrame({'movie_id' : ['dohwan'] *5, 'genre': my_favorite,'ratings':[30]*5})

if not data.isin({''})

### 5. CSR matrix를 직접 만들어 봅시다.

## 6. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

## 7. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 8. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 9. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

## 10. 회고